In [1]:
%load_ext tensorboard
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
from datetime import datetime
'''
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass
'''
%tensorboard --logdir logs/scalars/

tf.keras.backend.set_floatx('float64')

tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir= "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S"),
    histogram_freq=0,
    write_graph=True,
    write_grads=True
    )

checkpoint_path = "./training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
   checkpoint_path, verbose=1, save_weights_only=True,
   # Save weights, every epoch.
   save_freq='epoch')



#tf.get_logger().setLevel('INFO')

n_epochs = 30   # Number of optimization epochs
n_layers = 1    # Number of random layers
n_train = 200    # Size of the train dataset
n_test = 120     # Size of the test dataset
n_batches = 4     # Size of the batches

np.random.seed(0)           # Seed for NumPy random number generator
tf.random.set_seed(0)       # Seed for TensorFlow random number generator



mnist_dataset = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()



# Reduce dataset size
train_images = train_images[:n_train]
train_labels = train_labels[:n_train]
test_images = test_images[:n_test]
test_labels = test_labels[:n_test]

tf.config.get_visible_devices()


2024-05-11 21:44:52.050462: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-11 21:44:52.050515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-11 21:44:52.051601: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-11 21:44:52.057730: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-11 21:44:52.791335: W tensorflow/comp

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [104]:

# Normalize pixel values within 0 and 1
train_images = train_images / 255
test_images = test_images / 255

# Add extra dimension for convolution channels
train_images = np.array(train_images[..., tf.newaxis], requires_grad=False)
test_images = np.array(test_images[..., tf.newaxis], requires_grad=False)

n_qubits = 4

dev = qml.device("default.qubit", wires=n_qubits)
# Random circuit parameters
random_weights = np.random.uniform(high=2 * np.pi, size=(n_layers, n_qubits))

@qml.qnode(dev, interface='tf')
def qnode(inputs, weights):
    inputs *= np.pi
    # Encoding of 4 classical input values
    qml.AngleEmbedding(inputs, wires=range(n_qubits), rotation='Y')

    # Random quantum circuit
    RandomLayers(weights, wires=list(range(n_qubits)))

    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(n_qubits)]


@qml.qnode(dev, interface='tf')
def qnotnode(inputs):
    inputs *= np.pi
    # Encoding of 4 classical input values
    qml.AngleEmbedding(inputs, wires=range(n_qubits), rotation='Y')

    # Filter from arxiv.org/abs/2308.14930

    #qml.CNOT(wires=[1, 2])
    #qml.CNOT(wires=[0, 3])


    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(n_qubits)]

@qml.qnode(dev, interface='tf')
def qhadrandnode2(inputs):
    inputs *= np.pi
    # Encoding of 4 classical input values
    qml.AngleEmbedding(inputs, wires=range(n_qubits), rotation='Y')
    
    qml.Hadamard(wires=[1])
    qml.Hadamard(wires=[0])
    # Filter from arxiv.org/abs/2308.14930
    qml.ctrl(qml.Hadamard, control=1)(wires=2)
    qml.ctrl(qml.Hadamard, control=0)(wires=3)
    qml.CNOT(wires=[3, 0])
    qml.Hadamard(wires=[3])
    qml.ctrl(qml.Hadamard, control=3)(wires=2)

    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(n_qubits)]


In [105]:
class ConvQLayer(qml.qnn.KerasLayer):
    
    def call(self, inputs):

        out = tf.Variable(tf.zeros((n_batches, 14, 14, n_qubits)))
        for b in range(n_batches):
            # Loop over the coordinates of the top-left pixel of 2X2 squares
            for j in range(0, 28, 2):
                for k in range(0, 28, 2):
                    # Process a squared 2x2 region of the image with a quantum circuit
                    q_results = tf.stack(
                        [
                            inputs[b,j, k, 0],
                            inputs[b,j, k + 1, 0],
                            inputs[b,j + 1, k, 0],
                            inputs[b,j + 1, k + 1, 0]
                        ],
                        axis = 1
                    )
                    q_results = super().call(q_results)
                    # Assign expectation values to different channels of the output pixel (j/2, k/2)
                    for c in range(n_qubits):
                        out[b,j // 2, k // 2, c] = q_results[c]
        return out 

weight_shapes = {"weights": (n_layers, n_qubits)}

qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=[28,28,n_qubits])

qlayer.set_weights([random_weights]) 

qlayer.trainable = False

In [106]:

qnotlayer = qml.qnn.KerasLayer(qnotnode, {}, output_dim=[28,28,n_qubits])

qnotlayer.trainable = False

qhadrandlayer2 = qml.qnn.KerasLayer(qhadrandnode2, {}, output_dim=[28,28,n_qubits])

qhadrandlayer2.trainable = False

In [107]:

'''
def Q_Model():
    """Initializes and returns a custom Keras model
    which is ready to be trained."""
    model = keras.models.Sequential([
        qlayer,
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation="softmax")
    ])
    model.compile(
        optimizer='adam',
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

q_model = Q_Model()

q_history = q_model.fit(
    train_images,
    train_labels,
    validation_data=(test_images, test_labels),
    batch_size = n_batches,
    epochs=n_epochs,
    verbose=2, callbacks=[tensorboard_callback, cp_callback],
    shuffle=True
)
'''


'\ndef Q_Model():\n    """Initializes and returns a custom Keras model\n    which is ready to be trained."""\n    model = keras.models.Sequential([\n        qlayer,\n        keras.layers.Flatten(),\n        keras.layers.Dense(10, activation="softmax")\n    ])\n    model.compile(\n        optimizer=\'adam\',\n        loss="sparse_categorical_crossentropy",\n        metrics=["accuracy"],\n    )\n    return model\n\nq_model = Q_Model()\n\nq_history = q_model.fit(\n    train_images,\n    train_labels,\n    validation_data=(test_images, test_labels),\n    batch_size = n_batches,\n    epochs=n_epochs,\n    verbose=2, callbacks=[tensorboard_callback, cp_callback],\n    shuffle=True\n)\n'

In [108]:
'''
def Qnot_Model():
    """Initializes and returns a custom Keras model
    which is ready to be trained."""
    model = keras.models.Sequential([
        qnotlayer,
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation="softmax")
    ])
    model.compile(
        optimizer='adam',
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

qnot_model = Qnot_Model()

qnot_history = qnot_model.fit(
    train_images,
    train_labels,
    validation_data=(test_images, test_labels),
    batch_size = n_batches,
    epochs=n_epochs,
    verbose=2, callbacks=[tensorboard_callback, cp_callback],
    shuffle=True
)
'''

'\ndef Qnot_Model():\n    """Initializes and returns a custom Keras model\n    which is ready to be trained."""\n    model = keras.models.Sequential([\n        qnotlayer,\n        keras.layers.Flatten(),\n        keras.layers.Dense(10, activation="softmax")\n    ])\n    model.compile(\n        optimizer=\'adam\',\n        loss="sparse_categorical_crossentropy",\n        metrics=["accuracy"]\n    )\n    return model\n\nqnot_model = Qnot_Model()\n\nqnot_history = qnot_model.fit(\n    train_images,\n    train_labels,\n    validation_data=(test_images, test_labels),\n    batch_size = n_batches,\n    epochs=n_epochs,\n    verbose=2, callbacks=[tensorboard_callback, cp_callback],\n    shuffle=True\n)\n'

In [109]:

def Qhadrand2_Model():
    """Initializes and returns a custom Keras model
    which is ready to be trained."""
    model = keras.models.Sequential([
        qhadrandlayer2,
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation="softmax")
    ])
    model.compile(
        optimizer='adam',
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

qhadrand2_model = Qhadrand2_Model()

qhadrand2_history = qhadrand2_model.fit(
    train_images,
    train_labels,
    validation_data=(test_images, test_labels),
    batch_size = n_batches,
    epochs=n_epochs,
    verbose=2, callbacks=[tensorboard_callback, cp_callback],
    shuffle=True
)


Epoch 1/30



Epoch 1: saving model to ./training_2/cp-0001.ckpt
50/50 - 3s - loss: 2.4586 - accuracy: 0.2100 - val_loss: 1.8760 - val_accuracy: 0.4333 - 3s/epoch - 67ms/step
Epoch 2/30

Epoch 2: saving model to ./training_2/cp-0002.ckpt
50/50 - 3s - loss: 1.3141 - accuracy: 0.6300 - val_loss: 1.4977 - val_accuracy: 0.5417 - 3s/epoch - 64ms/step
Epoch 3/30

Epoch 3: saving model to ./training_2/cp-0003.ckpt
50/50 - 3s - loss: 0.9352 - accuracy: 0.7250 - val_loss: 1.5451 - val_accuracy: 0.4000 - 3s/epoch - 61ms/step
Epoch 4/30

Epoch 4: saving model to ./training_2/cp-0004.ckpt
50/50 - 3s - loss: 0.7296 - accuracy: 0.8050 - val_loss: 1.2442 - val_accuracy: 0.6333 - 3s/epoch - 66ms/step
Epoch 5/30

Epoch 5: saving model to ./training_2/cp-0005.ckpt
50/50 - 3s - loss: 0.5356 - accuracy: 0.8700 - val_loss: 1.1325 - val_accuracy: 0.6000 - 3s/epoch - 63ms/step
Epoch 6/30

Epoch 6: saving model to ./training_2/cp-0006.ckpt
50/50 - 3s - loss: 0.4542 - accuracy: 0.9100 - val_loss: 1.0248 - val_accuracy: 0.6